In [1]:
"""
Extract TreeMap (c.a. 2016) statistics in active fire detections

"""

import os, sys
import ee
import geemap
import time
import pandas as pd

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()

ee.Initialize(project='jfsp-aspen')

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Success")

Success


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [13]:
# Load the USFS TreeMap
treemap = ee.ImageCollection("USFS/GTAC/TreeMap/v2016")
print(f"TreeMap bands available for analysis:\n\n{treemap.first().bandNames().getInfo()}")

TreeMap bands available for analysis:

['ALSTK', 'BALIVE', 'CANOPYPCT', 'CARBON_D', 'CARBON_DWN', 'CARBON_L', 'DRYBIO_D', 'DRYBIO_L', 'FLDSZCD', 'FLDTYPCD', 'FORTYPCD', 'GSSTK', 'QMD_RMRS', 'SDIPCT_RMRS', 'STANDHT', 'STDSZCD', 'TPA_DEAD', 'TPA_LIVE', 'Value', 'VOLBFNET_L', 'VOLCFNET_D', 'VOLCFNET_L']


In [3]:
class_codes = treemap.first().get('FORTYPCD_class_values').getInfo()
class_names = treemap.first().get('FORTYPCD_class_names').getInfo()
code_to_name = dict(zip(class_codes, class_names))
species_df = pd.DataFrame(list(code_to_name.items()), columns=['species_code', 'species_name'])
species_df['species_code'] = species_df['species_code'].astype(int)
species_df.head()

,species_code,species_name
0,101,Jack pine
1,102,Red pine
2,103,Eastern white pine
3,104,Eastern white pine / eastern hemlock
4,105,Eastern hemlock


In [4]:
# Save this file out.
out_fp = os.path.join(projdir,'data/tabular/mod/treemap_fortypcd_species_mapping.csv')
df = pd.DataFrame(list(code_to_name.items()), columns=['FORTYPCD', 'SpeciesName'])
print(f"Dictionary saved to {out_fp}")

Dictionary saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/tabular/mod/treemap_fortypcd_species_mapping.csv


In [5]:
# Load the Active Fire Detections (AFD)
afds = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/vnp14img_geo_srm_pix_area_aspenfires')
print(afds.size().getInfo())
print(afds.first().propertyNames().getInfo())

51853
['Fire_ID', 'acq_date', 'daynight', 'Ig_Date', 'Last_Date', 'afdID', 'system:index']


## Calculate species-specific cover, live basal area (mean), and canopy percent (mean)

In [6]:
# Clean the property names before exporting reductions (just keep the ID)
afds_ = afds.select(['afdID'])

In [10]:
def species_histogram(ftr):
    image = treemap.select('FORTYPCD').first()
    hist = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ftr.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('FORTYPCD')

    hist_dict = ee.Dictionary(hist)
    flat_ftr = hist_dict.keys().iterate(
        lambda key, f: ee.Feature(f).set(ee.String(key), hist_dict.get(key)), ftr
    )
    return ee.Feature(flat_ftr)


# Apply this to the AFDs
fortypcd = afds_.map(species_histogram)

# Print a sample
sample = fortypcd.limit(1).getInfo()
properties = sample['features'][0]['properties']
df = pd.DataFrame([properties])
print("Sample DataFrame:")
print(df.head())

# Export to Asset/Drive
fortypcd = fortypcd.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

task = ee.batch.Export.table.toDrive(
    collection=fortypcd,
    description='vnp14img_treemap-fortypcd',
    fileFormat='CSV',
    folder='TreeMap'
)

# Start the export task
task.start()
print("Export to Google Drive started!")
# Monitor the task until it's finished
monitor_export(task, timeout=120) # print every X seconds

Sample DataFrame:
        182        185        201         221        261       267       281  \
0  37.52549  29.494118  23.129412  154.333333  13.666667  0.023529  1.690196   

        369  371      901        971  afdID  
0  9.082353    1  1.12549  31.352941  23291  
Export to Google Drive started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!


In [ ]:
# Species BALIVE and CANOPYPCT

In [16]:
def species_metrics(ftr):
    image = treemap.mosaic()
    # Get species histogram
    hist = image.select('FORTYPCD').reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ftr.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('FORTYPCD')

    hist_dict = ee.Dictionary(hist)
    
    # Initialize feature with species histogram keys and calculate averages
    def add_species_metrics(key, f):
        key_str = ee.String(key)
        # Mask the image by the species type
        masked_image = image.updateMask(image.select('FORTYPCD').eq(ee.Number.parse(key).toInt()))
        
        # Calculate average BALIVE and CANOPYPCT for the masked image within the polygon
        metrics = masked_image.select(['BALIVE', 'CANOPYPCT']).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ftr.geometry(),
            scale=30,
            maxPixels=1e13
        )
        
        # Add the averages to the feature with species-specific prefix
        f = ee.Feature(f).set(key_str.cat('_BALIVE_mean'), metrics.get('BALIVE'))
        f = f.set(key_str.cat('_CANOPYPCT_mean'), metrics.get('CANOPYPCT'))
        return f

    # Apply species metrics calculation to each species type in histogram
    flat_ftr = hist_dict.keys().iterate(add_species_metrics, ftr)
    
    return ee.Feature(flat_ftr)

fortypcd_metrics = afds_.map(species_metrics)

# Remove geometry column first
fortypcd_metrics = fortypcd_metrics.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=fortypcd_metrics,
    description='vnp14img_treemap-balive_canopypct',
    fileFormat='CSV', 
    fileNamePrefix='vnp14img_treemap-balive_canopypct',
    folder='TreeMap'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 120) # print every 120 seconds

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting fo

In [ ]:
# Load the exported CSV files.

In [ ]:
# Forest Type
fp = os.path.join(projdir,'data/earth-engine/exports/treemap/vnp14img_treemap-fortypcd.csv')
fortypcd = pd.read_csv(fp)
fortypcd.drop(columns=['system:index','.geo'], inplace=True)
fortypcd.columns

In [ ]:
# Pivot longer and join to get species name
fortypcd_l = fortypcd.melt(id_vars=['afdID'], var_name='species_code', value_name='count')
fortypcd_l['species_code'] = fortypcd_l['species_code'].astype(int)
# Drop 'NaN' counts
fortypcd_l.dropna(subset=['count'], inplace=True)
# Check on the results
fortypcd_l = fortypcd_l.sort_values(by=['afdID'])
fortypcd_l.head()

In [ ]:
fortypcd_l = fortypcd_l.merge(species_df, on='species_code', how='left')
fortypcd_l = fortypcd_l[['afdID','species_code','species_name','count']]
fortypcd_l['count'] = fortypcd_l['count'].round().astype(int)
fortypcd_l.head()

In [ ]:
# Calculate the total forest count and percent cover for each species
total_counts = fortypcd_l.groupby('afdID')['count'].sum().reset_index()
total_counts = total_counts.rename(columns={'count': 'total_count'})

# Join back to the data frame
fortypcd_l_pct = fortypcd_l.merge(total_counts, on='afdID', how='left')
fortypcd_l_pct['pct_cover'] = (fortypcd_l_pct['count'] / fortypcd_l_pct['total_count']) * 100
fortypcd_l_pct.head(12)

In [ ]:
# BALIVE and CANOPYPCT

In [ ]:
fp = os.path.join(projdir,'data/earth-engine/exports/treemap/vnp14img_treemap-balive_canopypct.csv')
stats_df = pd.read_csv(fp)
stats_df.drop(columns=['system:index','.geo'], inplace=True)
stats_df.columns

In [ ]:
len(stats_df['afdID'].unique())

In [ ]:
metrics_cols = [col for col in df.columns if '_BALIVE_mean' in col or '_CANOPYPCT_mean' in col]
print("Metrics columns selected for melting:", metrics_cols)
df_long = stats_df.melt(id_vars=['afdID'], value_vars=metrics_cols, 
                  var_name='metric', value_name='value')
len(df_long['afdID'].unique())

In [ ]:
df_long['species_code'] = df_long['metric'].str.extract(r'(\d+)_')[0].astype(int)
df_long['metric'] = df_long['metric'].str.extract(r'_(BALIVE|CANOPYPCT)_mean')[0]
df_long.head()

In [ ]:
stats_df_ = df_long.pivot_table(index=['afdID', 'species_code'], columns='metric', values='value').reset_index()
stats_df_['species_code'] = stats_df_['species_code'].astype(int)
stats_df_.head(12)

In [ ]:
# Export the TreeMap summary as a CSV

In [ ]:
# Join the two
fortypcd_l_pct['afdID'] = stats_df_['afdID'].astype(str)
stats_df_['afdID'] = stats_df_['afdID'].astype(str)
treemap_stats_df = fortypcd_l_pct.merge(stats_df_, on=['afdID','species_code'], how='left')
treemap_stats_df.head(15)

In [ ]:
# Save to file.
out_fp = os.path.join(projdir,'data/tabular/mod/AFD/vnp14img_treemap.csv')
treemap_stats_df.to_csv(out_fp)
print(f"Saved to: {out_fp}")

In [ ]:
gc.collect()